### mysql test

In [3]:
!echo 'select * from sqlite_master' | sqlite3 ../db/metrics.db

table|sqlite_sequence|sqlite_sequence|6|CREATE TABLE sqlite_sequence(name,seq)
table|metrics|metrics|2|CREATE TABLE metrics (id INTEGER PRIMARY KEY AUTOINCREMENT, test_id TEXT, version INTEGER, round INTEGER, model_id INTEGER, trial INTEGER, date DATETIME, metric TEXT, matched TEXT, matched_size INTEGER)
table|models|models|3|CREATE TABLE models (id INTEGER PRIMARY KEY AUTOINCREMENT, test_id TEXT, version INTEGER, model_id INTEGER, date DATETIME, model BLOB)
table|results|results|5|CREATE TABLE results (id INTEGER PRIMARY KEY AUTOINCREMENT, round INTEGER, metric TEXT, bonus INTEGER)
table|rndforest|rndforest|6568|CREATE TABLE rndforest (id INTEGER PRIMARY KEY AUTOINCREMENT,n_estimator INTEGER,data_length INTEGER,random_state INTEGER,rounds TEXT,matched_cnts TEXT,sum_val INTEGER,version TEXT)
table|version|version|8031|CREATE TABLE version(id INTEGER, name TXT, ver TXT)
table|rndforest_possi|rndforest_possi|9461|CREATE TABLE rndforest_possi(id INTEGER PRIMARY KEY AUTOINCREMENT, version 

In [4]:
!echo 'CREATE TABLE metrics (id INTEGER PRIMARY KEY AUTOINCREMENT, test_id TEXT, version INTEGER, round INTEGER, model_id INTEGER, trial INTEGER, date DATETIME, metric TEXT, matched TEXT, matched_size INTEGER)'| mysql -u swhors -h localhost -p

Enter password: 
ERROR 1045 (28000): Access denied for user 'swhors'@'localhost' (using password: YES)


In [ ]:
```
CREATE TABLE metrics (id int(16) NOT NULL AUTO_INCREMENT PRIMARY KEY, test_id varchar, version int, round int, model_id int, trial int, date datetime, metric varchar, matcher varchar, matched_size int);
```

In [ ]:
 CREATE TABLE metrics (id INT NOT NULL AUTO_INCREMENT PRIMARY KEY, test_id varchar(10), version INT, round INT, model_id INT, trial INT, date DATETIME, metric TEXT, matched TEXT, matched_size INT);

In [ ]:
CREATE TABLE models (id INT NOT NULL AUTO_INCREMENT PRIMARY KEY, test_id char(10), version INT, model_id INT, date DATETIME, model BLOB);

In [ ]:
CREATE TABLE results (id INT NOT NULL AUTO_INCREMENT PRIMARY KEY, round INT, metric TEXT, bonus INT);

In [ ]:
CREATE TABLE rndforest (id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,n_estimator INT,data_length INT,random_state INT,rounds TEXT,matched_cnts TEXT,sum_val INT,version char(10))

In [9]:
def to_mysql(table_name, cols_dict):
    import mysql.connector
    from contextlib import closing
 
    with closing(mysql.connector.connect(
        host="localhost",
        user="swhors",
        password="aq1234",
        database="metrics"
        )) as conn:
        with closing(conn.cursor()) as cursor:
            column_names = cols_dict.keys()
            column_values = cols_dict.values()
            column_names_str = ','.join(column_names)
            column_values_str = ""
            col_cnt = 0
            for column_value in column_values:
                column_values_str += column_value if column_value.isdigit() else f'\"{column_value}\"'
                col_cnt += 1
                if col_cnt < len(column_values):
                    column_values_str += ","
            query = f"insert into {table_name}({column_names_str}) values({column_values_str});"
            print
            cursor.execute(query)
        conn.commit()


In [ ]:
from tqdm import tqdm
rows = ! echo 'select * from results' | sqlite3 ../db/metrics.db
for row in tqdm(rows):
    cols = row.split('|')
    print(cols)
    cols_dicts = {
        'round': cols[1],
        'metric': cols[2],
        'bonus': cols[3]
    }
    to_mysql('results', cols_dicts)

In [16]:
from tqdm import tqdm
"""
,n_estimator INT,data_length INT,random_state INT,rounds TEXT,matched_cnts TEXT,sum_val INT,version char(10))
"""
rows = ! echo 'select * from rndforest' | sqlite3 ../db/metrics.db
for row in tqdm(rows):
    cols = row.split('|')
    # print(cols)
    cols_dicts = {
        'n_estimator': cols[1],
        'data_length': cols[2],
        'random_state': cols[3],
        'rounds': cols[4],
        'matched_cnts': cols[5],
        'sum_val': cols[6],
        'version': cols[7]
    }
    to_mysql('rndforest', cols_dicts)

 65%|██████████████████████████████████████████████████▍                          | 46705/71310 [31:34<16:38, 24.65it/s]


KeyboardInterrupt: 

In [ ]:
"""
rndforest_possi(id INTEGER PRIMARY KEY AUTOINCREMENT, version TEXT, random_state INTEGER, possibility FLOAT, max FLOAT)
"""
from tqdm import tqdm

rows = ! echo 'select * from rndforest_possi' | sqlite3 ../db/metrics.db
for row in tqdm(rows):
    cols = row.split('|')
    cols_dicts = {
        'version': cols[1],
        'random_state': cols[2],
        'possibility': cols[3],
        'max': cols[4],
    }
    to_mysql('rndforest', cols_dicts)